In [1]:
import sys

sys.path.append("../src")
import numpy as np
import pandas as pd

from ranking_sim import (
    process_raw_data,
    softmax_weights,
    simulate_round_based,
    brier_score,
)

In [2]:
np.random.seed(20250618)

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
INPUT_FOLDER = "../data/raw"

In [5]:
df = process_raw_data(f"{INPUT_FOLDER}/llm_and_human_leaderboard_overall.pkl")
df.head()

,id,source,direction,forecast_due_date,market_value_on_due_date,resolution_date,resolved_to,resolved,forecast,imputed,score,model,organization,question_type,horizon,question_id,market_forecast,question_market_discrepancy,median_question_market_discrepancy
0,1285,infer,(),2024-07-21,0.0163,2025-01-01,0.0,True,0.5000,False,0.250000,Claude-2.1 (scratchpad),Anthropic,market,NaN,infer-1285-nan-2024-07-21,0.0163,0.148065,0.106412
1,1286,infer,(),2024-07-21,0.0292,2025-01-01,0.0,True,0.0292,True,0.000853,Claude-2.1 (scratchpad),Anthropic,market,NaN,infer-1286-nan-2024-07-21,0.0292,0.195309,0.106412
2,1342,infer,(),2024-07-21,0.0088,2025-01-01,0.0,True,0.0088,True,0.000077,Claude-2.1 (scratchpad),Anthropic,market,NaN,infer-1342-nan-2024-07-21,0.0088,0.116829,0.106412
3,1353,infer,(),2024-07-21,0.4943,2024-12-01,0.0,True,0.4943,True,0.244332,Claude-2.1 (scratchpad),Anthropic,market,NaN,infer-1353-nan-2024-07-21,0.4943,0.031561,0.106412
5,1373,infer,(),2024-07-21,0.7474,2025-01-01,0.0,True,0.7474,True,0.558607,Claude-2.1 (scratchpad),Anthropic,market,NaN,infer-1373-nan-2024-07-21,0.7474,0.106195,0.106412


In [6]:
len(df["model"].unique())

141

In [7]:
len(df["question_id"].unique())

473

In [8]:
# Run a simulation to calibrate the change in Brier scores
# for the MODEL DRIFT scenario
res = []
for mm in range(1000):
    df_sim = simulate_round_based(
        df=df,
        n_rounds=2,
        questions_per_round=500,
        models_per_round_mean=30,
        ref_model="Naive Forecaster",
        skill_temperature=lambda round_id: -15.0 if round_id == 0 else 15.0,
        difficulty_temperature=None,
        model_persistence=0.0,
        fixed_models_per_round=False,
    )
    df_sim["score"] = brier_score(df_sim)
    df_temp = df_sim[["round_id", "score"]].groupby("round_id").mean().reset_index()
    brier_low_temp_sim = df_temp["score"].values[0]
    brier_high_temp_sim = df_temp["score"].values[-1]
    brier_delta_sim = brier_high_temp_sim - brier_low_temp_sim
    res.append(
        {
            "simulation_id": mm,
            "brier_low_temp": brier_low_temp_sim,
            "brier_high_temp": brier_high_temp_sim,
            "brier_delta": brier_delta_sim,
        }
    )
res = pd.DataFrame(res)
print(res["brier_delta"].mean())
res.head()

-0.06001343804726616


,simulation_id,brier_low_temp,brier_high_temp,brier_delta
0,0,0.247942,0.201690,-0.046252
1,1,0.240814,0.185489,-0.055325
2,2,0.243591,0.194413,-0.049177
3,3,0.241621,0.197096,-0.044524
4,4,0.236725,0.189929,-0.046796


In [9]:
# Run a simulation to calibrate the change in Brier scores
# for the QUESTION DRIFT scenario
res = []
for mm in range(1000):
    df_sim = simulate_round_based(
        df=df,
        n_rounds=2,
        questions_per_round=500,
        models_per_round_mean=30,
        ref_model="Naive Forecaster",
        skill_temperature=None,
        difficulty_temperature=lambda round_id: -5.0 if round_id == 0 else 5.0,
        model_persistence=0.0,
        fixed_models_per_round=False,
    )
    df_sim["score"] = brier_score(df_sim)
    df_temp = df_sim[["round_id", "score"]].groupby("round_id").mean().reset_index()
    brier_low_temp_sim = df_temp["score"].values[0]
    brier_high_temp_sim = df_temp["score"].values[-1]
    brier_delta_sim = brier_high_temp_sim - brier_low_temp_sim
    res.append(
        {
            "simulation_id": mm,
            "brier_low_temp": brier_low_temp_sim,
            "brier_high_temp": brier_high_temp_sim,
            "brier_delta": brier_delta_sim,
        }
    )
res = pd.DataFrame(res)
print(res["brier_delta"].mean())
res.head()

0.0888944432110465


,simulation_id,brier_low_temp,brier_high_temp,brier_delta
0,0,0.181794,0.257657,0.075862
1,1,0.149981,0.251407,0.101426
2,2,0.173055,0.256054,0.082999
3,3,0.175671,0.263982,0.088311
4,4,0.157614,0.253002,0.095388
